In [1]:
#|default_exp resample_img

# Resampling hyperspectral data

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import numpy as np
import pandas as pd
from spectral import BandResampler
import matplotlib.pyplot as plt


from hyperspectral_tools import const_vals as CONST

In [14]:
#|export

class Resample():

  def __init__(self,
               path_to_spectrum : str , #Path to a CSV file containing spectral bands as float values stored as strings. Ensure that only columns with spectral data have digit-only names strings.
               bands_center : list , # list of band center of specific sensor
               fwhm : list , # 
               ):
          
          #read the dataframe
          init_df = pd.read_csv(path_to_spectrum)
          # filter to get only spectral data and change columns type to numeric
          df = init_df.loc[:, init_df.columns[init_df.columns.str.isdigit()]]
          df.columns=pd.to_numeric(df.columns)
          self.df = df
          #save the non spectral data
          self.non_spectral_data =init_df.loc[:, init_df.columns[~init_df.columns.str.isdigit()]]

          #spectrum lists
          self.center_bands = bands_center
          self.fwhm = fwhm

          # Call resample_data method 
          self.resampled_df = self.resample_data()

          # Merge the non spectral data with spectral data
          self.resampled_final = self.merge_resample_with_original_data()


  def resample_data(self):
      resampler = BandResampler(self.df.columns.tolist(), 
                                   self.center_bands, 
                                   fwhm1=None, 
                                   fwhm2=self.fwhm)
      collect_res=[]
      for index,row in self.df.iterrows():
           tmp=self.df.iloc[index].values
           resampled_res=resampler(tmp)
           collect_res.append(resampled_res)

      self.resampled_data=pd.DataFrame(collect_res,columns=self.center_bands)

      return self.resampled_data
  
  def merge_resample_with_original_data(self):
       self.df_resampled_final =pd.concat([self.non_spectral_data,self.resampled_df],axis=1)

       return self.df_resampled_final
           
  



In [15]:
instance = Resample(path_to_spectrum = "data\original\processesed\cleansamples_209.csv",
                    bands_center = CONST.s2_center_sen,
                    fwhm = CONST.s2_fwhm_sen
                    )

test=instance.resampled_final
# test.dtypes

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\USER\AppData\Local\Temp\ipykernel_21684\283081024.py:1: SyntaxWarning: invalid escape sequence '\o'
  instance = Resample(path_to_spectrum = "data\original\processesed\cleansamples_209.csv",


,Unnamed: 0,Lon,Lat,TOC,clay,silt,sand,NI,442.7,492.4,559.8,664.6,704.1,740.5,782.8,864.7,945.1
0,1,54.736553,24.236353,0.22,0.0,1.0,98.0,0.025199,0.167390,0.213474,0.301375,0.403649,0.435667,0.461581,0.485040,0.506495,0.522874
1,2,54.712263,24.253125,0.23,0.0,0.0,100.0,0.029790,0.140747,0.182203,0.259326,0.347297,0.377320,0.402823,0.426922,0.449392,0.465502
2,3,54.628437,24.201236,0.54,7.0,2.0,71.0,0.021016,0.120756,0.160423,0.235413,0.302746,0.323227,0.341374,0.358821,0.372135,0.382613
3,4,54.637328,24.167353,0.06,0.0,1.0,74.0,0.012043,0.206508,0.252974,0.342654,0.421159,0.443675,0.462249,0.478891,0.489127,0.498079
4,5,54.871967,24.194968,5.97,0.0,1.0,94.0,0.116111,0.066941,0.090997,0.133236,0.206073,0.249886,0.292470,0.339111,0.413234,0.466920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,182,55.605843,24.689525,0.29,0.0,2.0,98.0,0.043041,0.087607,0.118148,0.189611,0.302176,0.335037,0.363015,0.389962,0.419415,0.445014
205,184,55.757064,24.567733,0.64,0.0,1.0,91.0,0.036654,0.090078,0.122434,0.200193,0.329732,0.363596,0.391099,0.417240,0.443891,0.468638
206,185,55.758205,24.566327,1.64,0.0,1.0,95.0,0.023077,0.105511,0.142010,0.230363,0.359417,0.388978,0.412330,0.433616,0.450980,0.468314
207,188,55.779780,24.440850,1.58,0.0,3.0,96.0,0.033870,0.122675,0.157790,0.225312,0.321491,0.354717,0.382239,0.407889,0.432313,0.448694


In [9]:
pd.concat([instance.non_spectral_data,instance.resampled_df],axis=1)


,Unnamed: 0,Lon,Lat,TOC,clay,silt,sand,NI,442.7,492.4,559.8,664.6,704.1,740.5,782.8,864.7,945.1
0,1,54.736553,24.236353,0.22,0.0,1.0,98.0,0.025199,0.167390,0.213474,0.301375,0.403649,0.435667,0.461581,0.485040,0.506495,0.522874
1,2,54.712263,24.253125,0.23,0.0,0.0,100.0,0.029790,0.140747,0.182203,0.259326,0.347297,0.377320,0.402823,0.426922,0.449392,0.465502
2,3,54.628437,24.201236,0.54,7.0,2.0,71.0,0.021016,0.120756,0.160423,0.235413,0.302746,0.323227,0.341374,0.358821,0.372135,0.382613
3,4,54.637328,24.167353,0.06,0.0,1.0,74.0,0.012043,0.206508,0.252974,0.342654,0.421159,0.443675,0.462249,0.478891,0.489127,0.498079
4,5,54.871967,24.194968,5.97,0.0,1.0,94.0,0.116111,0.066941,0.090997,0.133236,0.206073,0.249886,0.292470,0.339111,0.413234,0.466920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,182,55.605843,24.689525,0.29,0.0,2.0,98.0,0.043041,0.087607,0.118148,0.189611,0.302176,0.335037,0.363015,0.389962,0.419415,0.445014
205,184,55.757064,24.567733,0.64,0.0,1.0,91.0,0.036654,0.090078,0.122434,0.200193,0.329732,0.363596,0.391099,0.417240,0.443891,0.468638
206,185,55.758205,24.566327,1.64,0.0,1.0,95.0,0.023077,0.105511,0.142010,0.230363,0.359417,0.388978,0.412330,0.433616,0.450980,0.468314
207,188,55.779780,24.440850,1.58,0.0,3.0,96.0,0.033870,0.122675,0.157790,0.225312,0.321491,0.354717,0.382239,0.407889,0.432313,0.448694


In [ ]:
#resampler_ven=BandResampler(df.columns.tolist(), center_ven, fwhm1=None, fwhm2=fwhm_ven)